In [ ]:
!pip install --upgrade pip
!pip install transformers datasets evaluate accelerate peft wandb trl nltk

In [1]:
import json
import time
import torch
import transformers
import wandb
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
from transformers import BitsAndBytesConfig


# nltk 다운로드 (최초 1회)
nltk.download('punkt')

# wandb 초기화
wandb.init(project="inference_evaluation", name="4bit_quantization")

# corpus.json 파일 로드 (UTF-8 인코딩)
with open("./corpus.json", "r", encoding="utf-8") as f:
    evaluation_data = json.load(f)

# 10개만 추론
evaluation_data = evaluation_data[:10]

# 허깅페이스 방식으로 모델 및 파이프라인 초기화
model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
)


generator = transformers.pipeline(
    "text-generation",
    model=model_id,
    tokenizer=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    quantization_config=quantization_config  # 4비트 양자화를 적용합니다.
)
generator.model.eval()

# 시스템 프롬프트 정의
SYSTEM_PROMPT = (
    "You are a helpful AI assistant. Please answer the user's questions kindly. "
    "당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요."
)

# 채팅 템플릿을 적용하여 프롬프트를 생성하는 함수
def make_prompt(system_msg: str, user_msg: str) -> str:

    msg = f"""
다음은 유튜브 타임라인 요약 문서입니다. 이 내용을 분석하여 핵심 용어(KeyPoint)와 설명을 생성하세요.
설명을 생성할 때 영상 위주가 아닌 최대한 전문적이고 구체적으로 추가 설명해주세요. 저는 영상 내용 뿐 아니라 그 용어에 대한 내용을 알고 싶습니다.
중요하지 않더라도 알면 좋을 추가 핵심 용어도 더해주세요.

[타임라인 요약]:
{user_msg}

[가이드]:
- 필드는 `term`과 `description`을 포함하며, 결과는 여러 개의 키포인트 리스트로 구성됩니다.
- 반드시 JSON 형식에 맞춰 반환하세요.
- 다른 텍스트는 포함하지 마세요.

"""

    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": msg}
    ]
    prompt = generator.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return prompt

# 종료 토큰 설정
terminators = [
    generator.tokenizer.eos_token_id,
    generator.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]



# 평가 루프: corpus.json에 있는 각 예제에 대해 추론 수행
for idx, example in enumerate(evaluation_data):
    print(f"\n==== Example {idx+1} ====")

    # 사용자 입력과 프롬프트 생성
    user_input = example["input"]
    prompt = make_prompt(SYSTEM_PROMPT, user_input)

    # 입력 토큰 인코딩
    encoded_input = generator.tokenizer(prompt, return_tensors="pt")
    encoded_input = {k: v.to(generator.model.device) for k, v in encoded_input.items()}
    prompt_length = encoded_input["input_ids"].shape[1]

    # 속도 및 메모리 측정을 위한 시작 시간 기록 및 GPU 메모리 초기화
    start_time = time.time()
    if torch.cuda.is_available():
        torch.cuda.reset_peak_memory_stats(generator.model.device)

    # 모델 생성: 토큰별 score 반환 포함
    outputs = generator.model.generate(
        **encoded_input,
        max_new_tokens=2048,
        eos_token_id=terminators[0],
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        return_dict_in_generate=True,
        output_scores=True,
    )

    # GPU 동기화 후 종료 시간 기록
    if torch.cuda.is_available():
        torch.cuda.synchronize(generator.model.device)
    end_time = time.time()
    elapsed_time = end_time - start_time

    # GPU 메모리 사용량 측정 (Peak 메모리, MB 단위)
    if torch.cuda.is_available():
        peak_memory = torch.cuda.max_memory_allocated(generator.model.device) / (1024 ** 2)
    else:
        peak_memory = 0

    # 생성된 전체 토큰에서 프롬프트 이후의 candidate 토큰 추출
    generated_ids = outputs.sequences[0]
    candidate_ids = generated_ids[prompt_length:]
    candidate_text = generator.tokenizer.decode(candidate_ids, skip_special_tokens=True).strip()

    print("Candidate output:")
    print(candidate_text)
    print(f"Inference Time: {elapsed_time:.2f} seconds")
    print(f"Peak GPU Memory Usage: {peak_memory:.2f} MB")

    # 참조 텍스트: evaluation_data의 "output"을 JSON 문자열로 변환
    reference_text = json.dumps(example["output"], ensure_ascii=False, indent=2)

    # BLEU 점수 계산 (공백 기준 토큰화)
    candidate_tokens = candidate_text.split()
    reference_tokens = reference_text.split()
    bleu = sentence_bleu(
        [reference_tokens],
        candidate_tokens,
        smoothing_function=SmoothingFunction().method1
    )
    print(f"BLEU score: {bleu:.4f}")

    # 로그 확률 계산: 각 생성 토큰에 대해 log‑softmax 후 선택된 토큰의 로그 확률 합산
    log_score = 0.0
    for i, step_scores in enumerate(outputs.scores):
        log_probs = torch.log_softmax(step_scores, dim=-1)
        token_id = candidate_ids[i].unsqueeze(0)  # shape: (1,)
        token_log_prob = log_probs[0, token_id].item()
        log_score += token_log_prob
    print(f"Log probability score (sum): {log_score:.4f}")

    # wandb에 메트릭 로깅
    wandb.log({
        "example_index": idx,
        "BLEU_score": bleu,
        "Inference_time_sec": elapsed_time,
        "Peak_GPU_Memory_MB": peak_memory,
        "Log_probability_score": log_score
    })

wandb.finish()

[nltk_data] Downloading package punkt to /Users/kimhongil/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
wandb: Currently logged in as: iamkimhongil92 (iamkimhongil92-lumenasoft) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


[{'input': '이 영상은 유니티에서 2D 오브젝트를 만드는 방법을 소개하는 첫 번째 강좌입니다. 유니티 초보자를 위한 단계별 설치 및 프로젝트 생성 과정을 상세히 설명하며, 앞으로 구현할 플레이어 오브젝트 개발을 위한 준비 작업을 다룹니다. 이 영상에서는 유니티 설치, 에셋 가져오기, 오브젝트 만들기 및 필요한 컴포넌트 추가 과정을 중심으로 진행됩니다.\n\n- 안녕하세요! 골드메탈입니다. 유니티로 뱀사라이크 게임을 개발하는 첫 번째 시간을 시작합니다.\n- 유니티를 설치하고 기본 프로젝트를 생성하는 방법을 안내드립니다.\n- Unity.com.kr에서 유니티를 검색하여 공식 다운로드 페이지로 들어갑니다.\n- 윈도우용 다운로드 버튼을 클릭하여 유니티 허브 설치 파일을 받습니다.\n\n- 유니티 허브 설치 후, 계정으로 로그인해야 라이센스를 받을 수 있습니다.\n- 유니티 에디터를 설치하기 위해 허브의 설치 탭으로 가세요.\n- 본 영상에서는 2021.3 LTS 버전을 사용하는 방법을 설명합니다.\n- 여기서 안드로이드 빌드 지원 추가 설치도 잊지 말아야 합니다.\n\n- 새 프로젝트 버튼을 클릭하여 2D URP로 프로젝트를 생성합니다.\n- 프로젝트 이름과 경로를 설정하는 것은 매우 중요합니다. 한글 폴더는 피하세요.\n- 통합된 관리 시 프로젝트의 안정성을 높일 수 있습니다.\n\n- 이번 강좌를 위해 언데드 서바이버 에셋을 준비했습니다.\n- 골드메탈 스튜디오 페이지에서 해당 패키지를 다운로드하고 압축을 해제합니다.\n- 유니티 에디터에 에셋을 추가하는 방법: 더블 클릭하거나 드래그 앤 드롭을 사용하세요.\n\n- 플레이어 스프라이트를 사용해 게임 캐릭터를 만듭니다.\n- 스프라이트의 픽셀 퍼 유닛과 필터 모드 설정이 필요합니다.\n- 스프라이트 모드를 멀티플로 설정하여 필요한 부분만 자르세요.\n- 에디터의 스프라이트 설정을 통해 개별 이미지를 정확히 저장합니다.\n\n- 플레이어 오브젝트에 스프라이트 렌더러, 리지드 바디 2D, 캡슐 콜라이더 